In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import matplotlib.font_manager as fm
import torch
myfont = fm.FontProperties(fname='C:/Windows/Fonts/simsun.ttc')

import transformers
transformers.__version__
import pandas as pd

from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from transformers import ElectraTokenizer, ElectraModel, ElectraForMaskedLM

In [15]:
'''import xlxs'''

filename='Items_indi.xlsx'
df_N = pd.read_excel(filename)

df_N = df_N[['Verb','Condition','MaskedSentence']]
NL=len(df_N['MaskedSentence'])

prob_ba=np.zeros(NL,dtype = 'float64')
prob_bei=np.zeros(NL,dtype = 'float64')

top_k=60

def predict_masked_sent(text, top_k=10,print_en=True):
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)
    
    return_list=[]
    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        if print_en:
            print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))
        return_list.append({'token_str':predicted_token,'score':token_weight.numpy()})
    return return_list

def save_prob(model_name):
    
    prob_ba=np.zeros(NL,dtype = 'float64')  
    prob_bei=np.zeros(NL,dtype = 'float64')

    top_k=30

    for i in range(NL):
        #pre = predictor(dfsa2[i,j])
        pre = predict_masked_sent(df_N['MaskedSentence'][i], top_k=top_k,print_en=False)
        
        tmpscore=[tmp['score'] for tmp in pre if tmp['token_str']=='把']
        if tmpscore==[]:
            tmpscore=[0]
        prob_ba[i]=float(np.asarray(tmpscore))

        tmpscore=[tmp['score'] for tmp in pre if tmp['token_str']=='被']
        if tmpscore==[]:
            tmpscore=[0]
        prob_bei[i]=float(np.asarray(tmpscore))
        
    df_N['prob_ba']=prob_ba
    df_N['prob_bei']=prob_bei

    file_name = "Result_indi_{}.xlsx".format(model_name)

    df_N.to_excel(file_name)
    print(file_name,'saved')

1. BERT base chinese

In [16]:
model_folder="C:/Work/Classifier/HuggingFace/bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_folder)
model = BertForMaskedLM.from_pretrained(model_folder)
model.eval();

model_name = 'bert_base'
save_prob(model_name)

Some weights of the model checkpoint at C:/Work/Classifier/HuggingFace/bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Result_indi_bert_base.xlsx saved


2. roberta_base chinese

In [17]:
model_folder="C:/Work/Classifier/HuggingFace/chinese-roberta-wwm-ext"
tokenizer = BertTokenizer.from_pretrained(model_folder)
model = BertForMaskedLM.from_pretrained(model_folder)
model.eval();

model_name = 'roberta_base'
save_prob(model_name)


Some weights of the model checkpoint at C:/Work/Classifier/HuggingFace/chinese-roberta-wwm-ext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Result_indi_roberta_base.xlsx saved


3. electra_large chinese

In [18]:
model_folder="C:/Work/Classifier/HuggingFace/chinese-electra-large-generator"
tokenizer = ElectraTokenizer.from_pretrained(model_folder)
model = ElectraForMaskedLM.from_pretrained(model_folder)
model.eval();

model_name = 'electra_large'
save_prob(model_name)


Result_indi_electra_large.xlsx saved


4. ernie 1.0 Chinese

In [19]:
model_folder="C:/Work/Classifier/HuggingFace/ernie-1.0"
tokenizer = BertTokenizer.from_pretrained(model_folder)
model = BertForMaskedLM.from_pretrained(model_folder)
model.eval();

model_name = 'ernie'
save_prob(model_name)

Result_indi_ernie.xlsx saved


5 & 6. macbert base and large

In [20]:
model_folder="C:/Work/Classifier/HuggingFace/chinese-macbert-large"
tokenizer = BertTokenizer.from_pretrained(model_folder)
model = BertForMaskedLM.from_pretrained(model_folder)
model.eval();

model_name = 'macbert_large'
save_prob(model_name)


model_folder="C:/Work/Classifier/HuggingFace/chinese-macbert-base"
tokenizer = BertTokenizer.from_pretrained(model_folder)
model = BertForMaskedLM.from_pretrained(model_folder)
model.eval();

model_name = 'macbert_base'
save_prob(model_name)

Some weights of the model checkpoint at C:/Work/Classifier/HuggingFace/chinese-macbert-large were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Result_indi_macbert_large.xlsx saved


Some weights of the model checkpoint at C:/Work/Classifier/HuggingFace/chinese-macbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Result_indi_macbert_base.xlsx saved
